In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
# Download the dataset
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv')
raw_data = dataframe.values
dataframe.head()
#http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv

,-0.11252183,-2.8272038,-3.7738969,-4.3497511,-4.376041,-3.4749863,-2.1814082,-1.8182865,-1.2505219,-0.47749208,...,0.79216787,0.93354122,0.79695779,0.57862066,0.2577399,0.22807718,0.12343082,0.92528624,0.19313742,1.0
0,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,1.0
1,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,1.0
2,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,1.0
3,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,1.0
4,-1.507674,-3.574550,-4.478011,-4.408275,-3.321242,-2.105171,-1.481048,-1.301362,-0.498240,-0.286928,...,1.089068,0.983369,1.014124,0.952629,0.749326,1.007076,1.634990,1.493365,-0.783134,1.0


In [ ]:
# The last element contains the labels
labels = raw_data[:, -1]

# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

In [ ]:
min_val = tf.reduce_min(train_data)
max_val = tf.reduce_max(train_data)

train_data = (train_data - min_val) / (max_val - min_val)
test_data = (test_data - min_val) / (max_val - min_val)

train_data = tf.cast(train_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)

train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [ ]:
encoder_input = layers.Input(shape=(140,))
encoder = layers.Dense(64, activation="relu")(encoder_input)
encoder = layers.Dense(32, activation="relu")(encoder)
encoder = layers.Dense(16, activation="relu")(encoder)
encoder_output = layers.Dense(8, activation="relu")(encoder)

In [ ]:
decoder = layers.Dense(16, activation="relu")(encoder_output)
decoder = layers.Dense(32, activation="relu")(decoder)
decoder = layers.Dense(64, activation="relu")(decoder)
decoder_output = layers.Dense(140, activation="sigmoid")(decoder)

In [ ]:
autoencoder = Model(encoder_input, decoder_output)

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=100, 
          batch_size=512,
          validation_data=(test_data, test_data),shuffle=True)

In [ ]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

threshold = np.mean(train_loss) + 2*np.std(train_loss)

74/74 [==============================] - 0s 2ms/step


In [ ]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold)

In [ ]:
threshold

0.026452320627868176

In [ ]:
Novel_data_Normal=[-1.1008778e+00, -3.9968398e+00, -4.2858426e+00, -4.5065789e+00,
       -4.0223767e+00, -3.2343676e+00, -1.5661258e+00, -9.9225766e-01,
       -7.5467971e-01,  4.2321173e-02,  1.4895093e-01,  1.8352707e-01,
        2.9487608e-01,  1.9023267e-01,  2.3557500e-01,  2.5348712e-01,
        2.2174240e-01,  5.0233259e-02,  1.7804214e-01,  1.3956318e-01,
        4.6794433e-02,  4.3007139e-02,  1.0654432e-01,  1.2654003e-02,
        3.9948539e-03,  4.5724179e-02, -4.5999362e-02, -7.2666959e-02,
       -7.1077909e-02, -1.5386649e-01, -2.2725443e-01, -2.4926970e-01,
       -2.5348935e-01, -3.3283516e-01, -2.6432996e-01, -3.4582525e-01,
       -3.1078107e-01, -3.3415978e-01, -3.0617791e-01, -1.7456252e-01,
       -2.0616017e-01, -2.2447967e-01, -4.8683931e-02, -1.9621159e-02,
        9.4805186e-02,  1.1678639e-01,  1.8518592e-01,  2.3052577e-01,
        1.3802707e-01,  2.4442554e-01,  2.7169477e-01,  2.5900331e-01,
        2.6920166e-01,  1.2706463e-01,  2.0207339e-01,  2.5598343e-01,
        1.5311526e-01,  2.1021583e-01,  1.9108187e-01,  1.9092702e-01,
        2.1051006e-01,  2.0710764e-01,  2.0488835e-01,  2.0086474e-01,
        3.4691963e-01,  3.4800190e-01,  2.5453726e-01,  3.3197465e-01,
        3.5669030e-01,  3.4992982e-01,  4.3037963e-01,  4.6750163e-01,
        4.8606309e-01,  4.0318840e-01,  4.7895569e-01,  3.9739177e-01,
        4.6412003e-01,  4.5179539e-01,  3.6136121e-01,  3.7983090e-01,
        3.4278528e-01,  3.9910656e-01,  3.8750840e-01,  2.5570346e-01,
        2.8403408e-01,  2.8499257e-01,  2.5010809e-01,  1.7303626e-01,
        1.4942257e-01,  1.4168500e-01,  2.2226096e-01,  1.7284705e-01,
        1.5077932e-01,  1.7656601e-01,  2.8094204e-01,  4.8993901e-01,
        6.6088884e-01,  8.9319330e-01,  1.0269972e+00,  1.2023937e+00,
        1.5574096e+00,  1.8084277e+00,  2.1643459e+00,  2.0707467e+00,
        1.9036136e+00,  1.7644552e+00,  1.5077694e+00,  1.2934277e+00,
        8.9456215e-01,  5.7801575e-01,  2.4434305e-01, -2.8644345e-01,
       -5.1588134e-01, -7.3270694e-01, -8.3246547e-01, -8.0331806e-01,
       -8.3625248e-01, -7.7786456e-01, -7.7475301e-01, -7.3340394e-01,
       -7.2138631e-01, -8.3209517e-01, -7.1198159e-01, -7.5186730e-01,
       -7.5771960e-01, -8.5311967e-01, -7.6698841e-01, -6.8816064e-01,
       -5.1992349e-01,  3.9406169e-02,  5.6032676e-01,  5.3835583e-01,
        6.5688079e-01,  7.8748958e-01,  7.2404623e-01,  5.5578372e-01,
        4.7633335e-01,  7.7381971e-01,  1.1196209e+00, -1.4362499e+00]
        
Novel_data_Anomal=[8.9486742e-01, -3.8221580e-03, -7.5105205e-01, -1.3965429e+00,
       -1.7378376e+00, -2.1130203e+00, -2.2132096e+00, -2.2033471e+00,
       -2.0241906e+00, -1.5914581e+00, -1.0499022e+00, -5.7789108e-01,
       -3.1313740e-01, -2.0084404e-01, -1.8379899e-01,  1.6525057e-02,
        5.0921179e-01,  7.7751147e-01,  7.5067496e-01,  7.8843003e-01,
        7.7137524e-01,  7.3257374e-01,  6.7985885e-01,  6.2183591e-01,
        6.0469296e-01,  5.8129958e-01,  6.5998204e-01,  6.3413413e-01,
        6.3558912e-01,  7.6050608e-01,  7.6912011e-01,  7.3457027e-01,
        7.2108854e-01,  7.5040232e-01,  6.2612794e-01,  5.1689273e-01,
        5.4554218e-01,  5.0588773e-01,  5.2589320e-01,  4.9186076e-01,
        5.2836623e-01,  6.2282844e-01,  5.8039006e-01,  5.4824181e-01,
        5.0770483e-01,  5.1204975e-01,  4.0906504e-01,  3.1038144e-01,
        3.6845009e-01,  3.8137005e-01,  3.3419976e-01,  1.8130539e-01,
        2.0183031e-01,  2.9331710e-01,  2.7945872e-01,  2.9958438e-01,
        2.1474396e-01,  1.8456487e-01,  1.3197654e-01,  5.7778811e-02,
        1.8044803e-02, -7.1802363e-02,  5.5355480e-03,  6.6041000e-02,
        4.2362944e-02,  5.5254630e-02, -5.1268990e-03, -5.1986345e-02,
       -1.5207146e-02, -9.4131254e-03,  2.0942630e-02,  9.2754939e-02,
        7.3659051e-04, -7.1986495e-02,  7.0326647e-02,  9.1355005e-02,
        2.8342719e-02,  1.7460518e-01,  2.8679751e-01,  2.7782832e-01,
        3.1537371e-01,  3.1602348e-01,  3.6637824e-01,  4.4613065e-01,
        4.1105086e-01,  4.2461382e-01,  4.8312520e-01,  4.5440080e-01,
        4.6774525e-01,  5.6968959e-01,  5.4844061e-01,  5.2878176e-01,
        6.3875397e-01,  6.4354817e-01,  6.2231549e-01,  6.2276050e-01,
        5.7584803e-01,  5.6640109e-01,  5.6635399e-01,  5.7809064e-01,
        6.1300844e-01,  6.3278994e-01,  5.9456305e-01,  6.1561681e-01,
        7.2445263e-01,  7.1230460e-01,  6.7086495e-01,  7.1278444e-01,
        6.6167141e-01,  5.8293314e-01,  5.8568763e-01,  6.1013419e-01,
        6.9306756e-01,  6.8844189e-01,  6.1338458e-01,  6.0209334e-01,
        4.2122252e-01,  1.6604014e-01,  1.4347481e-01,  1.9632701e-01,
        1.6141046e-01,  1.2753130e-01, -1.8672703e-02, -2.3865283e-01,
       -3.7069690e-01, -5.9290109e-01, -6.3411579e-01, -4.4082747e-01,
       -5.1807030e-01, -7.0243851e-01, -8.9974628e-01, -1.4415163e+00,
       -1.8616119e+00, -2.3213812e+00, -2.8848769e+00, -3.5302460e+00,
       -4.4125149e+00, -3.9033308e+00, -3.5671064e+00, -1.5363381e+00]

In [ ]:
Novel_datapd=pd.DataFrame(Novel_data_Anomal)
ND=Novel_datapd.transpose()
ND=(ND+6.2808752)/13.6829783
ND=tf.cast(ND,tf.float32)
tf.print(predict(autoencoder,ND,threshold))

[0]


In [ ]:
import pickle 
pickle.dump(autoencoder,open('model.pkl','wb'))